In [9]:
import pandas as pd
import numpy as np
import os
from imblearn.combine import SMOTEENN
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [10]:
os.getcwd()

'E:\\Data_Science\\OpenClassRoom\\Projet7'

In [11]:

path = os.getcwd()
train_df = pd.read_csv(path+'\\full_impute_df.csv')

In [4]:
def preprocessing(df, imputer = False):
    #set the index as the id of the loan
    df.set_index('SK_ID_CURR')
    #wrong gender code
    df = df.drop(df.loc[df['CODE_GENDER'] == 'XNA'].index)
    #check if binary features are binary
    binary = [i for i in df.columns if 'FLAG' in i]
    for i in binary:
        if len(df[i].unique()) != 2:
            print('Column {} is not binary.'.format(i))
    #extract target
    if df['TARGET'].isna().sum() != 0:
        df = df.drop(df.loc[df['TARGET'].isna()])
    y = df['TARGET']
    df = df.drop(labels = 'TARGET', axis = 1)
    #OHE categorical features
    cat_df = df.select_dtypes(include = ['object'])
    num_df = df.select_dtypes(exclude = ['object'])
    ohe = OneHotEncoder(sparse = False)
    cat_df = ohe.fit_transform(cat_df)
    colnames = ohe.get_feature_names_out(ohe.feature_names_in_)
    cat_df = pd.DataFrame(cat_df, columns = colnames)
    #impute
    if imputer:
        num_df = imputer.fit_transform(num_df)
        colnames = imputer.feature_names_in_
        num_df = pd.DataFrame(num_df, columns = colnames)
        
    df = num_df.join(cat_df)

    colnames = df.columns
    return df, y, colnames 
    

In [12]:

df, df_y, colnames = preprocessing(train_df)

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(imp_df, imp_df_y)
smoteenn = SMOTEENN()
sampled_data = smoteenn.fit_resample(xtrain, ytrain)

In [ ]:
smoteen_df = pd.DataFrame(impute_df, columns = num_df.columns)
smoteen_df.to_csv('smoteen_df.csv', index = False)